# Llama 4 on Deep Infra

In [1]:
#IMPORTS & SETUP
import asyncio
import json
import os
import random
import nest_asyncio
from datetime import datetime
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm
from collections import Counter

# Apply patch for Notebook environments
nest_asyncio.apply()

# Load DeepInfra API key
keys = json.load(open("../apikeys.json"))
DI_KEY = keys["DeepInfra"]["key"]

In [2]:
# CONFIGURATION
# Maverick is the high-reasoning 400B model 
# (optimized for instruction following)
MODEL_ID = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8" 
# MODEL_ID = "meta-llama/Meta-Llama-4-400B-Instruct" # DeepInfra 2026 Path
# TOTAL_PROVERBS = 500
# CONCURRENCY_LIMIT = 50 # DeepInfra allows up to 200, but 50 is very stable
SAVE_INTERVAL = 20     # Auto-save every 20 proverbs
# OUTPUT_FILE = "responses-deepinfra-star.json"

client = AsyncOpenAI(
    api_key  = DI_KEY,
    base_url = "https://api.deepinfra.com/v1/openai"
)

A possible addition to the user prompt:
> Do not reference the internet, social media, or online forums explicitly. Be creative and original.

In [11]:
# List of mood modifiers to keep things fresh
MOODS = [
    "mystical and cryptic", "grumpy and cynical", "optimistic", "short and punchy", "poetic and flowing", "obsessed with hardware", "obsessed with social media", "acting like an old 1990s hacker", "hipster", "hippy", "conspiracy theorist"
]

async def fetch_proverb(semaphore, pbar):
    async with semaphore:
        # Pick a random mood for this specific request
        current_mood = random.choice(MOODS)
        
        try:
            response = await client.chat.completions.create(
                model=MODEL_ID,
                messages=[
                    {"role": "system", "content": f"You are a {current_mood} online denizen dispensing folk wisdom about modern life."},
                    {"role": "user", "content": "Say something about modern life, especially about life online. Single sentence, no quotes."}
                ],
                # REPETITION AVOIDANCE PARAMETERS
                temperature=1.2,
                frequency_penalty=0.8,
                presence_penalty=0.6,
                # extra_body={
                #     "min_p": 0.05,
                #     "frequency_penalty": 1.0,
                #     "presence_penalty": 0.8
                # },
                max_tokens=30
            )
            text = response.choices[0].message.content.strip()
            session_data.append({
                "text": text, 
                "timestamp": datetime.now().isoformat(),
                "mood": current_mood  # Track which mood produced it!
            })
            pbar.update(1)
        except Exception:
            pass

async def run_batch(batch_size, concurrency, filename):
    global session_data
    
    # Refresh local session_data from the target file if it exists
    if os.path.exists(filename):
        with open(filename, "r") as f:
            session_data = json.load(f)
        print(f"File '{filename}' loaded. Existing count: {len(session_data)}.")
    else:
        session_data = []
        print(f"Creating new file: '{filename}'")
    
    semaphore = asyncio.Semaphore(concurrency)
    print(f"Adding {batch_size} more proverbs...")
    
    with tqdm(total=batch_size) as pbar:
        tasks = [fetch_proverb(semaphore, pbar) for _ in range(batch_size)]
        await asyncio.gather(*tasks)
    
    # Save back to the specific filename provided
    with open(filename, "w") as f:
        json.dump(session_data, f, indent=4)
    
    print(f"Batch complete. Total in '{filename}': {len(session_data)}")

In [15]:
# EXECUTION
# Make changes to parameters without re-running the logic cell
MY_BATCH_SIZE = 1000
MAX_CONCURRENT = 25
OUTPUT_FILE = "r-di-11.json"

# Initialize list to hold session data
session_data = []

# Run the batch process
await run_batch(MY_BATCH_SIZE, MAX_CONCURRENT, OUTPUT_FILE)


File 'r-di-11.json' loaded. Existing count: 100.
Adding 1000 more proverbs...


100%|██████████| 1000/1000 [00:31<00:00, 32.02it/s]

Batch complete. Total in 'r-di-11.json': 1100


In [16]:
# Extract just the text from results
texts = [entry['text'] for entry in session_data]
counts = Counter(texts)

# Find proverbs that appeared more than once
duplicates = {text: count for text, count in counts.items() if count > 1}

print(f"Unique Proverbs: {len(counts)}")
print(f"Total Repetitions: {sum(duplicates.values()) - len(duplicates)}")
print("\nMost Frequent Repetitions:")
for text, count in sorted(duplicates.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"[{count}x] {text}")

Unique Proverbs: 1051
Total Repetitions: 49

Most Frequent Repetitions:
[8x] People spend so much time curating a highlight reel of their lives online that they forget to actually live them.
[5x] People spend so much time curating a highlight reel online that they forget to actually live their real lives.
[5x] People spend so much time curating online personas that they forget to actually live their real lives.
[4x] People spend so much time curating a highlight reel of their lives online that they forget to actually live the moments that matter.
[4x] People spend so much time curating a highlight reel of their lives online that they forget to actually live the moments that make up the reel.
[3x] People spend so much time curating their online personas they forget to actually live their real lives.
[3x] People these days are so caught up in curating their online personas that they've forgotten how to be genuine human beings in the real world.
[3x] People nowadays are so caught up in cu

In [ ]:

# Random Proverb
if session_data:
    # Pick a random entry from everything we've collected so far
    random_entry = random.choice(session_data)
    wisdom = random_entry['text']
    timestamp = datetime.fromisoformat(random_entry['timestamp']).strftime("%Y-%m-%d %H:%M")
    
    print("\n" + "="*50)
    print("📜 RECENT WISDOM FROM THE DIGITAL VOID:")
    print(f'"{wisdom}"')
    print("="*50)
    print(f"Recorded at: {timestamp} | Total Sage Knowledge: {len(session_data)} entries")
else:
    print("No wisdom found yet. Run the batch to begin.")